In [22]:
import time
import lattice_challenges_01

In [23]:
def subtract_row_in_triangular(input_matrix, line_index, by_index, k):
    for i in range(by_index+1):
        input_matrix[line_index, i] -= k*input_matrix[by_index, i]

def gram_schmidt_mu(v_i, v_j):
    return (v_i*v_j)/(v_j*v_j)

def mu(i, j, basis, gso):
    return gram_schmidt_mu(basis[i], gso[j])

def gram_schmidt_orthogonalization(basis):
    ret = matrix(RR, basis.nrows(), basis.ncols())
    for i in range(basis.nrows()):
        vector_being_orthogonalized = basis[i]
        for j in range(i):
            vector_being_orthogonalized -= gram_schmidt_mu(vector_being_orthogonalized, ret[j])*ret[j]
        ret[i] = vector_being_orthogonalized
    return ret

def LLL_usual_condition(input_basis, delta):
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and (delta - mu(i, i-1, basis, gso)**2)*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    return basis

def LLL_unusual_condition(input_basis, delta):
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    return basis

def add_line_to_triangular_matrix_diag(ret, current_row, ort_basis, diag_value):
    for i in range(current_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[current_row], ort_basis[i])
        ort_basis[current_row] -= t*ort_basis[i]
        ret[current_row, i] = t*ret[i, i]
    ret[current_row, current_row] = diag_value

def add_line_to_triangular_matrix(ret, current_row, ort_basis):
    for i in range(current_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[current_row], ort_basis[i])
        ort_basis[current_row] -= t*ort_basis[i]
        ret[current_row, i] = t*ret[i, i]
    ret[current_row, current_row] = sqrt(ort_basis[current_row]*ort_basis[current_row])

def triangularize_basis(basis, number_of_small_diagonal_elements):
    ret = matrix(RR, basis.nrows(), basis.nrows())
    ort_basis = matrix(RR, basis)
    current_row = 0
    while current_row < number_of_small_diagonal_elements:
        min_so_far = math.inf
        for m in range(current_row, ort_basis.nrows()):
            avec = ort_basis[m]
            for i in range(current_row):
                avec -= gram_schmidt_mu(avec, ort_basis[i])*ort_basis[i]
            t = sqrt(avec*avec)
            if t < min_so_far:
                min_so_far = t
                index_of_min_so_far = m
        if index_of_min_so_far != current_row:
            ort_basis[current_row], ort_basis[index_of_min_so_far] = ort_basis[index_of_min_so_far], ort_basis[current_row]
            basis[current_row], basis[index_of_min_so_far] = basis[index_of_min_so_far], basis[current_row]
        add_line_to_triangular_matrix_diag(ret, current_row, ort_basis, min_so_far)
        current_row += 1
    while current_row < ort_basis.nrows():
        add_line_to_triangular_matrix(ret, current_row, ort_basis)
        current_row += 1
    return ret

def reduce_triangular_basis(triangular_basis, parallel_basis):
    change = False
    for current_row in range(triangular_basis.nrows()-1, -1, -1):
        for m in range(current_row+1, triangular_basis.nrows()):
            t = round(triangular_basis[m, current_row]/triangular_basis[current_row, current_row])
            if t != 0:
                change = True
                subtract_row_in_triangular(triangular_basis, m, current_row, t)
                parallel_basis[m] -= t*parallel_basis[current_row]
    return change

def reduce_basis_using_triangular(input_basis, number_of_small_diagonal_elements):
    basis = copy(input_basis)
    change = True
    while change:
        triangular_basis = triangularize_basis(basis, number_of_small_diagonal_elements)
        change = reduce_triangular_basis(triangular_basis, basis)
    return basis

def swap_rows_in_triangular_basis(triangular_basis, i):
    bk = triangular_basis[i, i-1]
    ak = triangular_basis[i-1, i-1]
    bk1 = triangular_basis[i, i]
    triangular_basis[i, i-1] = sqrt(bk**2+bk1**2)
    triangular_basis[i-1, i-1] = ak*bk/triangular_basis[i, i-1]
    triangular_basis[i-1, i] = ak*bk1/triangular_basis[i, i-1]
    triangular_basis[i, i] = 0
    triangular_basis[i], triangular_basis[i-1] = triangular_basis[i-1], triangular_basis[i]
    for x in range(i+1, triangular_basis.nrows()):
        xa = triangular_basis[x, i-1]
        triangular_basis[x, i-1] = (xa*bk+triangular_basis[x, i]*bk1)/triangular_basis[i-1, i-1]
        triangular_basis[x, i] = (xa*ak-triangular_basis[x, i-1]*triangular_basis[i, i-1])/triangular_basis[i, i]

def LLL_usual_condition_using_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]**2*(delta-(triangular_basis[i, i-1]/triangular_basis[i-1, i-1])**2) > triangular_basis[i, i]**2:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

def LLL_unusual_condition_using_triangular(input_basis, delta):
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]*sqrt_delta > triangular_basis[i, i]:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis


In [24]:
def LLL_usual_condition_time(input_basis, delta):
    print("LLL_usual_condition_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and (delta - mu(i, i-1, basis, gso)**2)*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_usual_condition_time stops.")
    print()
    return basis

def LLL_unusual_condition_time(input_basis, delta):
    print("LLL_unusual_condition_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_unusual_condition_time stops.")
    print()
    return basis

def reduce_basis_using_triangular_time(input_basis, number_of_small_diagonal_elements):
    print("reduce_basis_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with number_of_small_diagonal_elements:")
    print(number_of_small_diagonal_elements)
    basis = copy(input_basis)
    start_time = time.time()
    change = True
    while change:
        triangular_basis = triangularize_basis(basis, number_of_small_diagonal_elements)
        change = reduce_triangular_basis(triangular_basis, basis)
    stop_time = time.time()
    print("measured times:")
    print((start_time, stop_time))
    print(stop_time - start_time)
    print("reduce_basis_using_triangular_time stops.")
    print()
    return basis

def LLL_usual_condition_using_triangular_time(input_basis, delta):
    print("LLL_usual_condition_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = input_basis.nrows()
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]**2*(delta-(triangular_basis[i, i-1]/triangular_basis[i-1, i-1])**2) > triangular_basis[i, i]**2:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_usual_condition_using_triangular_time stops.")
    print()
    return basis

def LLL_unusual_condition_using_triangular_time(input_basis, delta):
    print("LLL_unusual_condition_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]*sqrt_delta > triangular_basis[i, i]:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_unusual_condition_using_triangular_time stops.")
    print()
    return basis


In [25]:
def taxicab_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t += abs(input_matrix[i, j])
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_norms)

def chebyshev_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t2 = abs(input_matrix[i, j])
            if t2 > t:
                t = t2
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_norms)

def euclidean_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0.0
    sum_of_norms = 0.0
    sum_of_logarithms_of_squared_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0.0
        for j in range(input_matrix.ncols()):
            t += input_matrix[i, j]**2
        r = sqrt(t)
        sum_of_norms += r
        sum_of_logarithms_of_squared_norms += float(log(r))
        if r > longest_vector_norm:
            longest_vector_norm = r
        if r < shortest_vector_norm:
            shortest_vector_norm = r
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_squared_norms)

def all_properties(input_matrix):
    print(euclidean_properties(input_matrix))
    print(chebyshev_properties(input_matrix))
    print(taxicab_properties(input_matrix))


In [17]:
input_250 = matrix(lattice_challenges_01.challenge250())
output_250_LLL_usual_condition = LLL_usual_condition_time(input_250, 0.75)
output_250_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular_time(input_250, 0.75)
output_250_LLL_unusual_condition = LLL_unusual_condition_time(input_250, 0.74)
output_250_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular_time(input_250, 0.74)

all_properties(input_250)
all_properties(output_250_LLL_usual_condition)
all_properties(output_250_LLL_usual_condition_using_triangular)
all_properties(output_250_LLL_unusual_condition)
all_properties(output_250_LLL_unusual_condition_using_triangular)


LLL_usual_condition_time has been launched at time:
1605182261.8236303
on a matrix of dimension:
250 times 250
with delta:
0.750000000000000
measured times:
(1605182261.8238673, 1605182276.997887, 1605183896.4866204)
(15.174019575119019, 1634.6627531051636)
LLL_usual_condition_time stops.

LLL_usual_condition_using_triangular_time has been launched at time:
1605183896.497222
on a matrix of dimension:
250 times 250
with delta:
0.750000000000000
measured times:
(1605183896.4975376, 1605183913.1894968, 1605184376.9508047)
(16.691959142684937, 480.45326709747314)
LLL_usual_condition_using_triangular_time stops.

LLL_unusual_condition_time has been launched at time:
1605184376.9615543
on a matrix of dimension:
250 times 250
with delta:
0.740000000000000
measured times:
(1605184376.9618263, 1605184392.2203374, 1605186134.5223186)
(15.258511066436768, 1757.5604922771454)
LLL_unusual_condition_time stops.

LLL_unusual_condition_using_triangular_time has been launched at time:
1605186134.533969

In [13]:
all_properties(input_225)

(33.0000000000000, 131.202134128984, 21719.5488315009, 1012.77060005235)
(27, 33, 7130, 777.491172992671)
(33, 702, 103756, 1320.47528501228)


In [8]:
all_properties(output_200_LLL_usual_condition)
all_properties(output_200_LLL_usual_condition_using_triangular)
all_properties(output_200_LLL_unusual_condition)
all_properties(output_200_LLL_unusual_condition_using_triangular)

(29.3087017795057, 62.0886463051015, 9210.92769341581, 764.246648695798)
(11, 30, 3742, 582.502452842827)
(144, 318, 50119, 1103.23590964591)
(29.3087017795057, 62.0886463051015, 9193.25123974996, 763.862570195016)
(11, 30, 3739, 582.362690900452)
(144, 318, 50052, 1102.95550033279)
(22.6715680975093, 44.3734154646676, 6749.84801155265, 702.667137914127)
(8, 19, 2501, 502.386188226696)
(129, 290, 44355, 1078.98580169324)
(22.6715680975093, 44.3734154646676, 6750.61651166641, 702.691244894792)
(8, 19, 2497, 502.049715990075)
(129, 290, 44354, 1078.98093550359)


In [9]:
all_properties(input_200)

(30.0000000000000, 104.503588455134, 12990.8934862417, 825.343832594793)
(15, 30, 5331, 655.349824829410)
(30, 495, 55614, 1082.63564079353)


In [39]:
print(matrix([[258,0,0], [129,224,0], [0,113,999]]))
print(LLL_usual_condition_using_triangular(matrix([[258,0,0], [129,224,0], [0,113,999]]), 1.00))
print(LLL_usual_condition(matrix([[258,0,0], [129,224,0], [0,113,999]]), 1.00))
#print(reduce_basis_using_triangular(matrix([[258,0,0], [129,224,0], [0,113,999]]), 0))

[258   0   0]
[129 224   0]
[  0 113 999]
[ 258    0    0]
[-129  224    0]
[-129 -111  999]
[ 258    0    0]
[-129  224    0]
[-129 -111  999]


In [5]:
euclidean_properties(red)

(42.8135492572153, 79.2590688817374, 12449.5092404704, 824.941493693207)

In [6]:
euclidean_properties(matrix(lattice_challenges_01.challenge200()))

(30.0000000000000, 104.503588455134, 12990.8934862417, 825.343832594793)

In [7]:
reduce_basis_using_triangular(matrix([[257,0,0],[61,1,0],[0,2,1]]), 2)

[ 0  2  1]
[-4  5 -8]
[ 9  3 -7]

In [8]:
LLL_usual_condition_using_triangular(matrix([[257,0,0],[61,1,0],[0,2,1]]), 0.75)

[ 0  2  1]
[-4  5 -8]
[ 9  3 -7]

In [14]:
start_time = time.time()
Ap1 = LLL_usual_condition(matrix(lattice_challenges_01.challenge200()), 0.75)
end_time = time.time()

In [15]:
print(euclidean_properties(Ap1))
print(chebyshev_properties(Ap1))
print(taxicab_properties(Ap1))

(29.3087017795057, 62.0886463051015, 9210.92769341581, 764.246648695798)
(11, 30, 3742, 582.502452842827)
(144, 318, 50119, 1103.23590964591)


In [16]:
print(end_time - start_time)

793.998391866684


In [17]:
print(euclidean_properties(matrix(lattice_challenges_01.challenge200())))
print(chebyshev_properties(matrix(lattice_challenges_01.challenge200())))
print(taxicab_properties(matrix(lattice_challenges_01.challenge200())))

(30.0000000000000, 104.503588455134, 12990.8934862417, 825.343832594793)
(15, 30, 5331, 655.349824829410)
(30, 495, 55614, 1082.63564079353)
